In [1]:
import pandas as pd
import os
ref_data = pd.read_csv(os.path.join("/users/ziyzhang/topology-research/Nexullance_journal_data_gen/IT_cpp_compare/Nexullance_MP_APST4/Polarfly.csv"))
# ref_data = pd.read_csv(os.path.join("/users/ziyzhang/topology-research/Nexullance_journal_data_gen/IT_cpp_compare/Nexullance_OPT/DDF.csv"))

In [2]:
import sys
sys.path.append("/users/ziyzhang/topology-research/")
from topologies.Polarfly import PFtopo
import globals as gl
import numpy as np
# add the path to the Nexullance_IT c++ library
sys.path.append("/users/ziyzhang/topology-research/nexullance/IT_boost/build")
from Nexullance_IT_cpp import Nexullance_IT_interface


In [3]:
Cap_remote = 10 #GBps
Cap_local = 10 #GBps

In [4]:
def difference_against_opt(alpha, beta):

    sum_difference = 0

    configs = gl.pf_regular_configs[:1]
    
    for V, D in configs:
        EPR = (D+1)//2
        _network = PFtopo(V, D)
        arcs = _network.generate_graph_arcs()
        ASP, _ = _network.calculate_all_shortest_paths()
        ECMP_ASP = gl.ECMP(ASP)

        def run_for_traffic_pattern(_arcs, _traffic_pattern, _M_EPs, _alpha, _beta):
            remote_link_flows, local_link_flows = _network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, _M_EPs)
            max_remote_link_load = np.max(remote_link_flows)/Cap_remote
            max_local_link_load = np.max(local_link_flows)/Cap_local
            # adapt the traffic scaling factor to 10x saturation
            traffic_scaling = 10.0/max(max_local_link_load, max_remote_link_load)
            _M_EPs = traffic_scaling * _M_EPs
            remote_link_flows, local_link_flows = _network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, _M_EPs)
            max_remote_link_load = np.max(remote_link_flows)/Cap_remote
            max_local_link_load = np.max(local_link_flows)/Cap_local
            # run Nexullance_IT
            nexu_it = Nexullance_IT_interface(V, _arcs, gl.convert_M_EPs_to_M_R(_M_EPs, V, EPR), False)
            nexu_it.set_parameters(_alpha, _beta)
            nexu_it.run()
            Lremote_NEXU = nexu_it.get_max_link_load()
            Phi = gl.network_total_throughput(_M_EPs, Lremote_NEXU, max_local_link_load)
            return Phi/(V*EPR)

        traffic_pattern = "uniform"
        M_EPs = gl.generate_uniform_traffic_pattern(V, EPR)
        difference = (ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta))
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference
        
        traffic_pattern = "nearst-neighbour"
        M_EPs = gl.generate_diagonal_traffic_pattern(V, EPR, 1)
        difference = (ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta))
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference

        traffic_pattern = "shift_1"
        M_EPs = gl.generate_shift_traffic_pattern(V, EPR, 1)
        difference = (ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta))
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference

        traffic_pattern = "shift_half"
        M_EPs = gl.generate_half_shift_traffic_pattern(V, EPR)
        difference = (ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta))
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference

        traffic_pattern = "router-cluster"
        M_EPs = gl.generate_uniform_cluster_pattern(V, EPR, 4) # four clusters
        difference = (ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta))
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference
                
        traffic_pattern = "random-permute"
        M_EPs = gl.generate_random_permutation_pattern(V, EPR, 0)
        difference = round((ref_data[(ref_data["traffic"]==traffic_pattern) & (ref_data["V"]==V)]["Phi_per_EP"].values[0] - run_for_traffic_pattern(arcs, traffic_pattern, M_EPs, alpha, beta)), 3)
        assert difference >= 0, "Difference is negative"
        sum_difference -= difference

    return sum_difference

In [5]:
from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'alpha': (0.1, 10.0), 'beta': (0.0, 10.0)}
optimizer = BayesianOptimization(
    f=difference_against_opt,
    pbounds=pbounds,
    random_state=1,
)
optimizer.probe(
    params=[0.1, 7.0],
    lazy=True,
)
optimizer.probe(
    params=[1.0, 0.1],
    lazy=True,
)

/users/ziyzhang/miniconda3/envs/gt/lib/python3.12/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/users/ziyzhang/miniconda3/envs/gt/lib/python3.12/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/users/ziyzhang/miniconda3/envs/gt/lib/python3.12/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/users/ziyzhang/miniconda3/envs/gt/lib/python3.12/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [6]:
optimizer.maximize(
    init_points=10,
    n_iter=100,
)

|   iter    |  target   |   alpha   |   beta    |
-------------------------------------------------
step 2: no progress, terminating after0 attempts
| 1         | -1.286    | 0.1       | 7.0       |
step 2: no progress, terminating after10 attempts
step 2: no progress, terminating after0 attempts
step 2: no progress, terminating after4 attempts
step 2: no progress, terminating after0 attempts
step 2: no progress, terminating after4 attempts
step 2: no progress, terminating after5 attempts
step 2: no progress, terminating after3 attempts
| 2         | -5.962    | 1.0       | 0.1       |
step 2: no progress, terminating after0 attempts
| 3         | -1.383    | 4.229     | 7.203     |
step 2: no progress, terminating after10 attempts
step 2: no progress, terminating after10 attempts
step 2: no progress, terminating after0 attempts
step 2: no progress, terminating after9 attempts
step 2: no progress, terminating after3 attempts
| 4         | -4.005    | 0.1011    | 3.023     |
step 2: no 

In [7]:
optimizer.max

{'target': -1.0664335664335676,
 'params': {'alpha': 2.7623089332195714, 'beta': 3.1577691172637135}}